<a href="https://colab.research.google.com/github/taksaphat/gas_emission_model/blob/main/Model_final_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import all necessary modules

In [1]:
#@title
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import tensorflow as tf
import pandas as pd
import seaborn as sns  # to plot images and statical version of data
import numpy as np
import matplotlib.pyplot as plt
import requests
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression

from sklearn.metrics import r2_score
from sklearn import metrics
import csv

get dataset from google drive

In [2]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
data = drive.CreateFile({'id':"1toEuurwskgECiHDSQhOYG6swQRfoTFzD"})  
data.GetContentFile('emission data_fix.csv')  

In [4]:
dataset = pd.read_csv('emission data_fix.csv')

In [5]:
fieldnames = ['Year2018', 'Year2019','Year2020','Year2021','Year2022','Year2023','Year2024','Year2025','Year2026','Year2027','Year2028','Year2029','Year2030','Year2031','Year2032','Country','Maxacc','bestdf']
with open('countries_prediction_proto.csv', 'w', encoding='UTF8') as f:
  writer = csv.writer(f)

  # write the header
  writer.writerow(fieldnames)

  for i in range(229):
    #define degree of freedom (df) for polynomial regression
    degree = 1
    #define empty array for append r2score value each loop
    countAcc = []
    xdata = np.arange(1998,2018)
    ydata = dataset.loc[i, ['1998', '1999', '2000', '2001', '2002', '2003',
          '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
          '2015', '2016', '2017']].values
    
    x = xdata
    y = ydata
    x = xdata.reshape(-1,1)
    #split 80/20
    while True:
      X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42,shuffle=False)
      lm=make_pipeline(PolynomialFeatures(degree),LinearRegression())
      lm.fit(X_train,y_train)
      # test accuracy of trained data (R2_Score)
      score_pred = lm.predict(X_test)
      score_real = y_test
      np.ravel(lm.predict(X_test), order='C')
      np.ravel(y_test, order='C')
      r2print = r2_score(score_real,score_pred)
      countAcc.append(r2print)
      #check accuracy at current df and get maximum one
      max_value = np.max(countAcc)
      if (r2print >= 0.9 and degree >= 3):
        break
      if (degree < 10):
        degree = degree + 1
      else:
        break
    # prediction part
    if(max_value == 1 or max_value <= 0.9):
      continue
    ghgGrow = np.arange(2018, 2032+1)
    ghgGrow = ghgGrow.reshape(-1,1)
    ghgGrow_pred = lm.predict(ghgGrow)
    if(ghgGrow_pred[-1] < 0):
      continue
    strCountry = str(dataset.loc[i,['Country']].values)
    ghgGrow_w = ghgGrow_pred.tolist()
    ghgGrow_w.append(strCountry)
    ghgGrow_w.append(max_value)
    ghgGrow_w.append(np.argmax(countAcc)+1)
    # write result in csv
    writer.writerow(ghgGrow_w)